In [1]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [2]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [3]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [4]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [5]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [6]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [7]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [8]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[8.4539e+02, 1.1000e+00],
         [9.3070e+01, 1.5000e-01],
         [9.5086e+01, 6.5600e-01],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.5620e+02, 1.1000e+00],
         [4.1039e+01, 1.5400e-03],
         [4.4049e+01, 5.6500e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[5.8911e+02, 1.1000e+00],
         [1.1721e+02, 9.6000e-02],
         [1.5507e+02, 9.8000e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[4.4714e+02, 1.1000e+00],
         [1.8608e+02, 1.8322e-01],
         [1.9305e+02, 1.2119e-01],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[4.2920e+02, 1.1000e+00],
         [1.0808e+02, 2.

In [9]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256,  79, 256, 256, 256, 256, 256, 256, 256])


In [10]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [11]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [12]:
tmp_original['precursor_mz']

tensor([845.3900, 356.1969, 589.1100, 518.3250, 777.8614, 755.2390, 229.0900,
        326.1751, 317.0650, 447.1392, 429.2000, 300.1473])

In [13]:
tmp_original['adduct'], 

(['[M+Na]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+'],)

In [14]:
tmp_original['mol_freq']

tensor([490.,   3.,   9.,  17.,  37.,   8., 266.,  64.,  19.,  25., 123., 262.])

In [15]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [16]:
sum(tmp_original['labels'])

tensor(12)

In [17]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1103, 1359, 1615, 1871, 2127, 2383, 2639])

In [18]:
len(tmp_original['labels'])

2895

In [19]:
tmp_original['batch_ptr']

tensor([256, 256, 256, 256,  79, 256, 256, 256, 256, 256, 256, 256])

In [20]:
sum(tmp_original['batch_ptr'])

tensor(2895)

In [21]:
len(tmp_original['candidates'])

2895

In [22]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [23]:
len(tmp_original['candidates_smiles'])

2895

# MSnRetrieval

In [24]:
from massspecgym.featurize import SpectrumFeaturizer

In [25]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [26]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [27]:
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.json"

In [31]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/min_sample_trees.mgf"


In [28]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 16476
MSnRetrievalDataset length: 16476
Using 8 processes for precomputation.


In [30]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [31]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 12536
Val dataset size: 1952


In [90]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

[PROFILE] __getitem__ took 0.039332 seconds
[PROFILE] __getitem__ took 0.027529 seconds
[PROFILE] __getitem__ took 0.024225 seconds
[PROFILE] __getitem__ took 0.025832 seconds
[PROFILE] __getitem__ took 0.022352 seconds
[PROFILE] __getitem__ took 0.023395 seconds
[PROFILE] __getitem__ took 0.022568 seconds
[PROFILE] __getitem__ took 0.022822 seconds
[PROFILE] __getitem__ took 0.023671 seconds
[PROFILE] __getitem__ took 0.025100 seconds
[PROFILE] __getitem__ took 0.021958 seconds
[PROFILE] __getitem__ took 0.025602 seconds
[PROFILE] collate_fn took 0.026229 seconds
{'spec': DataBatch(x=[145, 1039], edge_index=[2, 266], batch=[145], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([405.1391, 278.1400, 493.1250, 500.1994, 466.2449, 301.1434, 371.

In [91]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256,  40, 256, 256, 256, 256, 256, 256, 256, 256],
       dtype=torch.int32)


In [77]:
def print_key_types(obj, obj_name):
    print(f"Data types in {obj_name}:")
    for key, value in obj.items():
        print(f"  {key}: {type(value)}")

# Print data types for both objects
print_key_types(tmp_original, "tmp_original")
print_key_types(tmp_msn, "tmp_msn")

Data types in tmp_original:
  spec: <class 'torch.Tensor'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  identifier: <class 'list'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>
Data types in tmp_msn:
  spec: <class 'abc.DataBatch'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  identifier: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [36]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [37]:
tmp_msn['spec'] 

DataBatch(x=[110, 1039], edge_index=[2, 196], batch=[110], ptr=[13])

In [38]:
tmp_msn['precursor_mz']

tensor([1335.6216,  575.2235,  728.3852,  272.0994,  397.1922,  927.3129,
         238.0993,  275.1076,  406.1220,  422.1657,  359.1212,  311.2006])

In [39]:
tmp_msn['mol_freq']

tensor([2., 2., 1., 1., 3., 2., 1., 1., 1., 1., 1., 2.])

In [40]:
tmp_msn['identifier']

['0083946_0000000',
 '0012304_0000000',
 '0014804_0000000',
 '0080235_0000000',
 '0071811_0000000',
 '0009920_0000000',
 '0069306_0000000',
 '0068912_0000000',
 '0022248_0000000',
 '0017028_0000000',
 '0092975_0000000',
 '0011887_0000000']

In [41]:
tmp_msn['adduct']

['[M+H]+',
 '[M+H]+',
 '[M+NH4]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H-H2O]+',
 '[M+H]+',
 '[M+H]+',
 '[M+NH4]+',
 '[M+H]+',
 '[M+H-2H2O]+']

In [42]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [43]:
tmp_msn['smiles']

['CC1C2C(C(C(O1)OC3C(C(COC3OC(=O)C45CCC(CC4C6=CCC7C(C6(CC5O)C)(CCC8C7(CC(C(C8(C)CO)OC9C(C(C(C(O9)CO)O)O)OC1C(C(C(CO1)OC(=O)CC(CC(=O)O2)(C)O)O)O)O)C)C)(C)C)O)O)O)OC1C(C(C(CO1)O)O)O',
 'COC(=O)C1=CO[C@H]([C@@H]([C@@H]1C[C@H]2C3=C(C[C@H](N2)C(=O)O)C4=CC=CC=C4N3)C=C)O[C@H]5[C@@H]([C@H]([C@@H]([C@H](O5)CO)O)O)O',
 'CC1C(C(C(C(O1)OC2CCC3(C(C2)CCC4C3CCC5(C4(CCC5C6=CC(=O)OC6)O)C)C=O)O)OC)OC7C(C(C(C(O7)CO)O)O)O',
 'C1=CC(=CC=C1CNC2=NC=NC3=C2C=C(C=C3)F)F',
 'C[C@H](C1=CC2=C(C=C1)N(CC2)C(=O)C3CCOCC3)NC(=O)C4=CC=C(C=C4)F',
 'C[C@H]1[C@@H]([C@H]([C@H]([C@@H](O1)O[C@H]2[C@@H](O[C@H]([C@@H]([C@@H]2O)O)O[C@@H]3[C@H]([C@H](O[C@@H]([C@H]3OC(=O)/C=C/C4=CC=C(C=C4)O)CO)O[C@]5([C@H]([C@@H]([C@H](O5)CO)O)OC(=O)/C=C/C6=CC=C(C=C6)O)CO)O)C)O)O)O',
 'CC(C(=O)C1=CC(=CC=C1)Cl)NC(C)(C)CO',
 'CN(C)CCN(C)CCC1=CC(=C(C=C1)Cl)Cl',
 'COC1=CC=C(C=C1)CC2C(=O)N(/C(=N\\N=C/C3=CC=CO3)/S2)C4=CC=CC=C4',
 'COC(=O)C1=CO[C@H]([C@H]2[C@@]1(CC=C2CO)O)O[C@H]3[C@@H]([C@H]([C@@H]([C@H](O3)CO)O)O)O',
 'C1=CC=C(C=C1)C(CC(=O)NC2=NC3=CC=CC

In [44]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [45]:
sum(tmp_msn['labels'])

tensor(12)

In [46]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,    5,   93,  185,  441,  697,  715,  971, 1227, 1483, 1739, 1995])

In [47]:
len(tmp_msn['labels'])

2251

In [48]:
tmp_msn['batch_ptr']

tensor([  5,  88,  92, 256, 256,  18, 256, 256, 256, 256, 256, 256],
       dtype=torch.int32)

In [49]:
sum(tmp_msn['batch_ptr'])

tensor(2251, dtype=torch.int32)

In [50]:
len(tmp_msn['candidates'])

2251

In [51]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [52]:
tmp_msn.keys() == tmp_original.keys()

True

In [53]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [54]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

False

In [55]:
tmp_original['batch_ptr'].dtype

torch.int64

In [56]:
tmp_msn['batch_ptr'].dtype

torch.int32

In [57]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(2251, dtype=torch.int32), 2251)

In [58]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(2895), 2895)

In [59]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [60]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [61]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [62]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [63]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [64]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [65]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [66]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [67]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [68]:
len(tmp_msn['candidates_smiles'])

2251